<a href="https://colab.research.google.com/github/Santosh-Gupta/NaturalLanguageRecommendations/blob/master/notebooks/build_index_and_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --id "10LV9QbZOkUyOzR4nh8hxesoKJhpmvpM9" # citation vectors
!gdown --id 1-1HED-B-HtuZR9kNtybz9sypVpk2A_fX   # TitlesAbstractsEmbedIds
!wget 'https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar'
!tar -xvf 'scibert_scivocab_uncased.tar'

Downloading...
From: https://drive.google.com/uc?id=10LV9QbZOkUyOzR4nh8hxesoKJhpmvpM9
To: /content/CitationSimilarityVectors106Epochs.npy
2.59GB [00:21, 120MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1wIRsAApaE2L7E1fjnDOSSVBG1fY-LT9i
To: /content/tfworld.zip
411MB [00:04, 86.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-1HED-B-HtuZR9kNtybz9sypVpk2A_fX
To: /content/TitlesAbstractsEmbedIds.json.gzip
397MB [00:02, 150MB/s] 
--2019-12-28 11:27:06--  https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.144.44
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.144.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 442460160 (422M) [application/x-tar]
Saving to: ‘scibert_scivocab_uncased.tar’

scibert_scivocab_un 100%[===================>] 421.96M  44.3MB/s    in 9.0s    

2019-12-28

In [3]:
!pip install transformers --quiet

     |████████████████████████████████| 450kB 3.0MB/s 
     |████████████████████████████████| 860kB 7.9MB/s 
     |████████████████████████████████| 1.0MB 11.7MB/s 


In [4]:
%tensorflow_version 2.x
import numpy as np
import tensorflow as tf
from time import time
from tqdm import tqdm_notebook as tqdm
from transformers import BertTokenizer
import pandas as pd
from pprint import pprint

print('TensorFlow:', tf.__version__)

TensorFlow: 2.1.0-rc1


In [5]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  ['10.11.245.194:8470']
INFO:tensorflow:Initializing the TPU system: 10.11.245.194:8470


INFO:tensorflow:Initializing the TPU system: 10.11.245.194:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [6]:
workers = ['/job:worker/replica:0/task:0/device:TPU:'+str(i) for i in range(8)]
workers

['/job:worker/replica:0/task:0/device:TPU:0',
 '/job:worker/replica:0/task:0/device:TPU:1',
 '/job:worker/replica:0/task:0/device:TPU:2',
 '/job:worker/replica:0/task:0/device:TPU:3',
 '/job:worker/replica:0/task:0/device:TPU:4',
 '/job:worker/replica:0/task:0/device:TPU:5',
 '/job:worker/replica:0/task:0/device:TPU:6',
 '/job:worker/replica:0/task:0/device:TPU:7']

In [0]:
class Index:
    def __init__(self, embeddings, worker):
        self.embeddings = tf.math.l2_normalize(embeddings, axis=1)
        self.worker = worker

    @tf.function
    def search(self, query_vector):
      with tf.device(worker):
        dot_product = tf.reduce_sum(tf.multiply(self.embeddings, query_vector), axis=1)
        distances = 1 - dot_product
        sorted_indices =  tf.argsort(distances)
        nearest_distances = tf.gather(distances, sorted_indices)
        return nearest_distances[:20], sorted_indices[:20]

In [8]:
citations_embeddings = np.load('CitationSimilarityVectors106Epochs.npy')
citations_embeddings.shape

(1262996, 512)

In [9]:
# Discarding last 4 vectors to make number of vectors divisible by 8
citations_embeddings = np.split(citations_embeddings[:-4], 8, axis=0)
vecs_per_index = citations_embeddings[0].shape[0]
print('Vectors per index :', vecs_per_index)

Vectors per index : 157874


In [10]:
## Place 1/8 of total embeddings on each TPU core
indices = []
for i, worker in enumerate(workers):
  with tf.device(worker):
    print('Building index with {} vectors on {}'.format(citations_embeddings[i].shape[0],worker))
    indices.append(Index(citations_embeddings[i], worker))

Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:0
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:1
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:2
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:3
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:4
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:5
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:6
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:7


In [0]:
def search(xq, top_k=10):
  D, I = [], []
  for i in range(8):
    print('Search running on {}'.format(indices[i].worker))
    d, idx = indices[i].search(xq)
    D.extend(d.numpy())
    I.extend(i*vecs_per_index + idx.numpy())

  id_sorted = np.argsort(D)[:top_k]
  D = np.array(D)[id_sorted]
  I = np.array(I)[id_sorted]
  return D, I

In [12]:
n = 0
split = 6 # [0, 7] . # Pick nth vector from given split
actual_n = vecs_per_index*split + n

xq = tf.nn.l2_normalize(citations_embeddings[split][n])
xq = tf.reshape(xq, [1, -1])

s = time()
D, I = search(xq, top_k=5)
e = time()

print('\nActual ID   :', actual_n)
print('Result ID   :', I[0], '\n')

print('Neighbours   :', I )
print('Distances   :', np.round(D, 4))
print('\nTime taken  :', np.round(e-s, 2), 'secs')
# First search runs slow, because tf.function traces the function
# only for the first invocation, following invocations should run fine

Search running on /job:worker/replica:0/task:0/device:TPU:0
Search running on /job:worker/replica:0/task:0/device:TPU:1
Search running on /job:worker/replica:0/task:0/device:TPU:2
Search running on /job:worker/replica:0/task:0/device:TPU:3
Search running on /job:worker/replica:0/task:0/device:TPU:4
Search running on /job:worker/replica:0/task:0/device:TPU:5
Search running on /job:worker/replica:0/task:0/device:TPU:6
Search running on /job:worker/replica:0/task:0/device:TPU:7

Actual ID   : 947244
Result ID   : 947244 

Neighbours   : [ 947244  556804  202475 1033955  104991]
Distances   : [0.     0.2717 0.4    0.6742 0.6773]

Time taken  : 13.85 secs


In [0]:
model = tf.saved_model.load('gs://tfworld/saved_models')
tokenizer = BertTokenizer(vocab_file='scibert_scivocab_uncased/vocab.txt')

df = pd.read_json('TitlesAbstractsEmbedIds.json.gzip', compression = 'gzip')
embed2Title = pd.Series(df['title'].values,index=df['EmbeddingID']).to_dict()
embed2Abstract = pd.Series(df['paperAbstract'].values,index=df['EmbeddingID']).to_dict()

In [14]:
embed_id = 70
title = embed2Title[embed_id]
abstract = embed2Abstract[embed_id]

abstract_encoded = tokenizer.encode(abstract, max_length=512, pad_to_max_length=True)
abstract_encoded = tf.constant(abstract_encoded, dtype=tf.int32)[None, :]
print('Title : ')
pprint(title)
print('\nAbstract : ')
pprint(abstract)

Title : 
'Embedded operating systems energy overhead'

Abstract : 
("In this paper, a flow of characterization of embedded operating system's "
 'energy consumption is presented. The objective is to determine the energy '
 'overhead of the services of the embedded OS, we interest particularly on the '
 'context switch service. The modeling is based on measurements on the '
 'hardware platform OMAP35x EVM board, running Linux omap. Based on the '
 'analysis results, a relationship between energy overhead and a set of '
 'hardware and software parameters is established.')


In [16]:
s = time()
bert_output = model(abstract_encoded)
xq = tf.nn.l2_normalize(bert_output, axis=1)
e_p = time()

D, I = search(xq, top_k=5)
e_s = time()

print('\n'*2)
print('*'*80)
for i in range(len(I)):
  print('Title : ')
  pprint(embed2Title[I[i]])
  print('*'*80)

print('\n'*2)
print('Neighbours       :', I )
print('Distances        :', np.round(D, 4))
print('Prediction time  :', np.round(e_p-s, 3), 'secs')
print('Search time      :', np.round(e_s-e_p, 3), 'secs')
print('Total time       :', np.round(e_s - s, 3), 'secs')

Search running on /job:worker/replica:0/task:0/device:TPU:0
Search running on /job:worker/replica:0/task:0/device:TPU:1
Search running on /job:worker/replica:0/task:0/device:TPU:2
Search running on /job:worker/replica:0/task:0/device:TPU:3
Search running on /job:worker/replica:0/task:0/device:TPU:4
Search running on /job:worker/replica:0/task:0/device:TPU:5
Search running on /job:worker/replica:0/task:0/device:TPU:6
Search running on /job:worker/replica:0/task:0/device:TPU:7



********************************************************************************
Title : 
'Compiler-directed high-level energy estimation and optimization'
********************************************************************************
Title : 
('Power Measurement of Software Programs on Computers With Multiple I/O '
 'Components')
********************************************************************************
Title : 
'Data partitioning for maximal scratchpad usage'
*******************************************